# Исследование надёжности заёмщиков

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Обработка-ошибок-в-данных-и-дубликатов" data-toc-modified-id="Обработка-ошибок-в-данных-и-дубликатов-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обработка ошибок в данных и дубликатов</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Удаление-неинформативных-столбцов" data-toc-modified-id="Удаление-неинформативных-столбцов-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Удаление неинформативных столбцов</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответьте-на-вопросы" data-toc-modified-id="Ответьте-на-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответы на вопросы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - ответить на вопросы:

1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

1. Обзор данных. Данные о поведении пользователей мы получим из файла `/datasets/data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.
2. Предобработка данных. Мы проверим данные на ошибки и поищем возможность исправить самые критичные ошибки данных.
3. Ответы на вопросы.

**Описание данных**

- `children` — количество детей в семье 
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита.

## Обзор данных

In [1]:
# Импортируем библиотеку pandas
import pandas as pd

In [2]:
# Сохраним в переменную data таблицу data.csv
data = pd.read_csv('/datasets/data.csv')
# Получим общую информацию о data
data.info()
data.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20721,0,-933.244403,23,среднее,1,женат / замужем,0,M,компаньон,0,164154.911697,строительство жилой недвижимости
15368,2,-243.007947,40,среднее,1,женат / замужем,0,M,компаньон,0,115102.509418,покупка жилой недвижимости
17427,0,-1407.728133,36,высшее,0,женат / замужем,0,F,компаньон,0,325390.989659,операции с недвижимостью
367,0,-683.274502,57,Среднее,1,вдовец / вдова,2,F,сотрудник,0,74253.433737,на покупку подержанного автомобиля
4580,0,364886.037535,62,среднее,1,женат / замужем,0,F,пенсионер,0,70934.571734,заняться образованием


In [3]:
# Получим более подробную информацию о data
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

После открытия и сохранения данных в переменной `data`, можно сделать следующие выводы:
1. Размеры таблицы - 12 колонок и 21525 строк. Наименование столбцов соответствует змеиному регистру.
2. Столбцы `days_employed` и `total_income` имеют только по 19351 значений. Остальные значения утеряны или не указаны.
3. Столбецы `children`, `days_employed`, `dob_years` имеют аномальные значения.
4. Столбецы `children`, `days_employed` имеют отрицательные значения, что неверно.

## Предобработка данных

### Обработка пропусков

На этапе обзора таблицы мы увидели, что столбцы `days_employed` и `total_income` имеют пропущенные значения. Определим количество пропущенных значений и наличие взаимосвязи со значениями в других столбцах, после чего примем решение по обработке пропуском.

Проверим связаны ли отсутствующие значения в столбцах `days_employed` и `total_income`.

In [4]:
print(f'Количество отсутствующих значений в столбце days_employed = {data["days_employed"].isna().sum()} или {data["days_employed"].isna().sum()/len(data):.2%} данных.')
print(f'Количество отсутствующих значений в столбце total_income = {data["total_income"].isna().sum()} или {data["total_income"].isna().sum()/len(data):.2%} данных.')
data[data['days_employed'].isna()]['total_income'].value_counts()

Количество отсутствующих значений в столбце days_employed = 2174 или 10.10% данных.
Количество отсутствующих значений в столбце total_income = 2174 или 10.10% данных.


Series([], Name: total_income, dtype: int64)

Пропуски в данных встречаются одновременно в двух столбцах. При этом потеряно 10.10% данных, что много. Проверим на наличие взаимосвязи между отсутствующими данными и значениями в других столбцах.

In [5]:
# Создадим функцию для упрощения анализа
def nan_analysis(column):
    print(f'Распределение {column} среди отсутствующих значений:')
    display(data[data['days_employed'].isna()][column].value_counts())

In [6]:
nan_analysis('children')

Распределение children среди отсутствующих значений:


 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

Закономерности со столбцом `children нет`. Однако в столбце children есть значения (-1, 20), которые скорее всего неверные. Обработкой этих значений займемся позже.

In [7]:
nan_analysis('dob_years')

Распределение dob_years среди отсутствующих значений:


34    69
40    66
31    65
42    65
35    64
36    63
47    59
41    59
30    58
28    57
57    56
58    56
54    55
38    54
56    54
37    53
52    53
39    51
33    51
50    51
51    50
45    50
49    50
29    50
43    50
46    48
55    48
48    46
53    44
44    44
60    39
61    38
62    38
64    37
32    37
27    36
23    36
26    35
59    34
63    29
25    23
24    21
66    20
65    20
21    18
22    17
67    16
0     10
68     9
69     5
20     5
71     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64

В 10 значениях возраст клиентов равен 0. Обработкой этих значений займемся позже.

In [8]:
nan_analysis('education')

Распределение education среди отсутствующих значений:


среднее                1408
высшее                  496
СРЕДНЕЕ                  67
Среднее                  65
неоконченное высшее      55
Высшее                   25
ВЫСШЕЕ                   23
начальное                19
Неоконченное высшее       7
НЕОКОНЧЕННОЕ ВЫСШЕЕ       7
Начальное                 1
НАЧАЛЬНОЕ                 1
Name: education, dtype: int64

Мы видим большое количество дубликатов. Приведем все значения к нижнему регистру и проверим распределение снова.

In [9]:
data['education'] = data['education'].str.lower()

In [10]:
nan_analysis('education')

Распределение education среди отсутствующих значений:


среднее                1540
высшее                  544
неоконченное высшее      69
начальное                21
Name: education, dtype: int64

Закономерности со столбцом `education` нет. 

In [11]:
nan_analysis('education_id')

Распределение education_id среди отсутствующих значений:


1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64

Распределение в столбцах `education` и `education_id` одинаковое. Проверим распределение на всех данные позже. Закономерности со столбцом `education_id` нет. 

In [12]:
nan_analysis('family_status')

Распределение family_status среди отсутствующих значений:


женат / замужем          1237
гражданский брак          442
Не женат / не замужем     288
в разводе                 112
вдовец / вдова             95
Name: family_status, dtype: int64

Наведем порядок в данных и приведем все значения к нижнему регистру.

In [13]:
data['family_status'] = data['family_status'].str.lower()

In [14]:
nan_analysis('family_status')

Распределение family_status среди отсутствующих значений:


женат / замужем          1237
гражданский брак          442
не женат / не замужем     288
в разводе                 112
вдовец / вдова             95
Name: family_status, dtype: int64

Теперь все в порядке. Закономерности со столбцом `family_status` нет.

In [15]:
nan_analysis('family_status_id')

Распределение family_status_id среди отсутствующих значений:


0    1237
1     442
4     288
3     112
2      95
Name: family_status_id, dtype: int64

Распределение в столбцах `family_status` и `family_status_id` одинаковое. Проверим распределение на всех данные позже. Закономерности со столбцом `family_status_id` нет.

In [16]:
nan_analysis('gender')

Распределение gender среди отсутствующих значений:


F    1484
M     690
Name: gender, dtype: int64

Закономерности со столбцом `gender` нет.

In [17]:
nan_analysis('income_type')

Распределение income_type среди отсутствующих значений:


сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Закономерности со столбцом `income_type` нет.

In [18]:
nan_analysis('debt')

Распределение debt среди отсутствующих значений:


0    2004
1     170
Name: debt, dtype: int64

Закономерности со столбцом `debt` нет. Более 92% клиентов не имели задолженностей по кредитам.

In [19]:
nan_analysis('purpose')

Распределение purpose среди отсутствующих значений:


на проведение свадьбы                     92
сыграть свадьбу                           81
свадьба                                   76
строительство собственной недвижимости    75
операции с жильем                         74
покупка недвижимости                      72
операции со своей недвижимостью           71
покупка жилья для семьи                   71
ремонт жилью                              70
операции с коммерческой недвижимостью     70
покупка коммерческой недвижимости         67
покупка жилья для сдачи                   65
недвижимость                              62
операции с недвижимостью                  61
покупка жилой недвижимости                61
жилье                                     60
строительство недвижимости                59
автомобили                                57
заняться высшим образованием              56
заняться образованием                     55
сделка с подержанным автомобилем          54
на покупку своего автомобиля              53
покупка жи

Закономерности со столбцом purpose нет. Однако в столбце purpose имеются дубликаты (различные формулировки одной и той же цели кредита), которые мы позже обработаем.

In [20]:
# Определим количество отрицательных значений в столбце days_employed
print(f'Количество отрицательных значений в столбце days_employed = {data[data["days_employed"] < 0]["days_employed"].count()} или {data[data["days_employed"] < 0]["days_employed"].count()/len(data):.2%} данных.')

Количество отрицательных значений в столбце days_employed = 15906 или 73.90% данных.


Перед анализом распределений значений в столбцах `total_income`, `days_employed` произведем замену отрицательных значений столбца `days_employed` на положительные с помощью `abs()`.

In [21]:
# Заменим отрицательные значения на положительные
data['days_employed'] = abs(data['days_employed'])

In [22]:
# Посмотрим на распределение значений в столбцах с отсутствующими значениями
data[['total_income', 'days_employed']].describe()

,total_income,days_employed
count,1.935100e+04,19351.000000
mean,1.674223e+05,66914.728907
std,1.029716e+05,139030.880527
min,2.066726e+04,24.141633
25%,1.030532e+05,927.009265
50%,1.450179e+05,2194.220567
75%,2.034351e+05,5537.882441
max,2.265604e+06,401755.400475


В столбах `total_income` и `days_employed` имеется большое количество выбросов с большими значениями. Об этом говорят медианные и средние значения столбцов.

Взаимосвязи с содержимым других столбцов нет, поэтому можно сделать вывод, что произошел системный сбой при заполнении данных о работе, или клиенты не хотели указывать данные о работе.

Для столбца `total_income`, который необходим для анализа и ответа на вопросы, произведем замену отсутствующих значений на медианное значение в зависимости от типа занятости.

In [23]:
# Создадим функцию для замены отсутствующих значений total_income
def total_income_median(row): 
    median_value = data[data['income_type'] == row['income_type']]['total_income'].median()
    return median_value

In [24]:
# Применим функцию
data.loc[data['total_income'].isna(), 'total_income'] = data[data['total_income'].isna()].apply(total_income_median, axis=1)

In [25]:
# Проверим результат замены
data[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Для столбца `days_employed`, который не учавствует в анализе и ответе на поставленные вопросы, произведем замену отсутствующих значений на медианное значение (условие задания) по возрасту. Если медианное значение более 50 лет (18250 дней), то поделим его на 24, т.к. есть гипотеза, что большие значения получились из-за того, что данные указаны в часах.

In [26]:
# Создадим функцию для замены отсутствующих значений days_employed
def days_employed_median(row): 
    median_value = data[data['dob_years'] == row['dob_years']]['days_employed'].median()
    if median_value > 18250:
        median_value = median_value / 24
    return median_value

In [27]:
# Применим функцию
data.loc[data['days_employed'].isna(), 'days_employed'] = data[data['days_employed'].isna()].apply(days_employed_median, axis=1)

In [28]:
# Проверим результат замены
data[data['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [29]:
# Посмотрим на распределение после замены
data[['total_income', 'days_employed']].describe()

,total_income,days_employed
count,2.152500e+04,21525.000000
mean,1.652253e+05,60613.246473
std,9.804367e+04,133165.975701
min,2.066726e+04,24.141633
25%,1.077982e+05,1007.368820
50%,1.425944e+05,2170.569747
75%,1.955499e+05,5489.396756
max,2.265604e+06,401755.400475


В столбце `days_employed` все еще остались слишком большие значения, которые необходимо обработать. Анализ показал, что аномальные значения скорее всего связаны с указанием данных в часах . Будем обрабатывать строки в которых days_employed > (dob_years - 16)* 365 (работать можно с 16 лет). Значения в данных строках разделим на 24.

In [30]:
# Создадим счетчик
count = 0
# Заменим аномальные значения
for i in data.index:
    if data.loc[i, 'days_employed'] > (data.loc[i, 'dob_years'] - 16) * 365:
        data.loc[i, 'days_employed'] = data.loc[i, 'days_employed'] / 24
        count += 1
print(f'Заменено {count} значений или {count/len(data):.2%} данных.')

Заменено 3618 значений или 16.81% данных.


In [31]:
# Посмотрим на распределение после обработки
data[['days_employed']].describe()

,days_employed
count,21525.000000
mean,4585.562124
std,5313.905911
min,4.540293
25%,976.884186
50%,2127.664054
75%,5369.839985
max,16739.808353


Теперь данные выгледят корректно.

**Вывод**

В столбцах `days_employed` и `total_income` имеется около 10.10 % потерянных данных, которые были обработаны следующим образом:

- для столбца days_employed, который не учавствует в  ответе на поставленные вопросы, мы произвели замену отрицательных значений на положительные, т.к. трудовой стаж не может быть отрицательным. Для отсутствующих значений произвели замену на медианные значения по возрасту. Слишком большие значения, которые не могут быть реальными, были разделены на 24 (гипотеза - указан стаж в часах).
- для столбца `total_income`, который необходим для ответа на вопросы, мы заменили отсутствующие значения на медианные значения в зависимости от типа занятости.

Данные были заменены медианными значениями из-за наличия выбросов, которые оказывают влияние среднее значение.

Точную причину отсутствия данных в столбцах `days_employed` и `total_income` установить невозможно, но среди возможных причин следующие:

- системный сбой - неверные действия при сборе данных, создании или обработке таблиц;
- нежелание клиентов заполнять данные о трудовом стаже и доходе.

В остальных столбцах таблицы есть "недочеты", которые будут устранены в следующих шагах.

### Обработка ошибок в данных и дубликатов

Как мы выяснили в пункт 2.1 некоторые столбцы таблицы data имеют дубликаты и ошибки. Самое время их устранить.

In [32]:
# Создадим функцию для упрощения анализа столбцов
def column_analysis(column):
    print(f'Распределение значений в столбце {column}:')
    display(data[column].value_counts())

In [33]:
column_analysis('children')

Распределение значений в столбце children:


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Количество детей не может быть отрицательным. Значение -1 заменим на 1. 20 детей тоже "аномальное" число для этого столба, возможно имеет место опечатка. Заменим значение 20 на 2.

In [34]:
# Замена значений
data.loc[data['children'] == -1, 'children'] = 1
data.loc[data['children'] == 20, 'children'] = 2

In [35]:
# Проверим результат замены
column_analysis('children')

Распределение значений в столбце children:


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Замена произведена верно.

In [36]:
column_analysis('dob_years')

Распределение значений в столбце dob_years:


35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Имеется 101 нулевое значение. Проверим на наличие взаимосвязи со значениями в других столбцах.

In [37]:
data[data['dob_years'] == 0]['children'].value_counts()

0    69
1    16
2    14
3     2
Name: children, dtype: int64

In [38]:
data[data['dob_years'] == 0]['days_employed'].value_counts()

73.293251     11
58.729174      1
178.696434     1
28.412807      1
334.811970     1
              ..
51.902427      1
42.438553      1
133.697197     1
42.308144      1
247.686118     1
Name: days_employed, Length: 91, dtype: int64

In [39]:
data[data['dob_years'] == 0]['education'].value_counts()

среднее                64
высшее                 35
неоконченное высшее     2
Name: education, dtype: int64

In [40]:
data[data['dob_years'] == 0]['family_status'].value_counts()

женат / замужем          49
гражданский брак         21
не женат / не замужем    16
в разводе                10
вдовец / вдова            5
Name: family_status, dtype: int64

In [41]:
data[data['dob_years'] == 0]['gender'].value_counts()

F    72
M    29
Name: gender, dtype: int64

In [42]:
data[data['dob_years'] == 0]['income_type'].value_counts()

сотрудник      55
компаньон      20
пенсионер      20
госслужащий     6
Name: income_type, dtype: int64

In [43]:
data[data['dob_years'] == 0]['debt'].value_counts()

0    93
1     8
Name: debt, dtype: int64

In [44]:
print('Минимальное значение:', data[data['dob_years'] == 0]['total_income'].min())
print('Максимальное значение:', data[data['dob_years'] == 0]['total_income'].max())
data[data['dob_years'] == 0]['total_income'].value_counts()

Минимальное значение: 34974.45036571415
Максимальное значение: 386373.63695802353


142594.396847    5
118514.486412    3
172357.950966    2
263121.074528    1
71291.522491     1
                ..
386373.636958    1
287328.538328    1
181510.868867    1
313949.845188    1
287173.027278    1
Name: total_income, Length: 94, dtype: int64

In [45]:
data[data['dob_years'] == 0]['purpose'].value_counts()

покупка жилья                             6
операции с жильем                         6
жилье                                     6
свадьба                                   5
на покупку автомобиля                     5
сыграть свадьбу                           5
на проведение свадьбы                     4
недвижимость                              4
автомобиль                                4
покупка недвижимости                      3
заняться высшим образованием              3
строительство жилой недвижимости          3
на покупку своего автомобиля              3
свой автомобиль                           3
образование                               3
покупка жилой недвижимости                3
покупка коммерческой недвижимости         3
высшее образование                        3
операции с недвижимостью                  3
получение дополнительного образования     2
ремонт жилью                              2
автомобили                                2
покупка жилья для сдачи         

Взаимосвязи с другими столбцами нет, поэтому восстановить данные не получится. Этот столбец не нужен для ответа на вопросы, поэтому оставим нулевые значения без изменений. Это позволит определить проблемные данные в будущем.

In [46]:
column_analysis('education')

Распределение значений в столбце education:


среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Значения в данном столбце мы уже ранее привели к нижнему регистру. Дополнительные действия не требуются.

In [47]:
column_analysis('education_id')

Распределение значений в столбце education_id:


1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Значения полностью сопоставимы со значениями в столбце `education`. Необходимости в обоих столбцах нет, так как они дублируют друг друга. Один из них мы позже удалим.

In [48]:
column_analysis('family_status')

Распределение значений в столбце family_status:


женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Значения в данном столбце мы уже ранее привели к нижнему регистру. Дополнительные действия не требуются.

In [49]:
column_analysis('family_status_id')

Распределение значений в столбце family_status_id:


0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Значения полностью сопоставимы со значениями в столбце `family_status`. Необходимости в обоих столбцах нет, так как они дублируют друг друга. Один из них мы позже удалим.

In [50]:
column_analysis('gender')

Распределение значений в столбце gender:


F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце gender имеется одно значение XNA. Его удаление никак не повлияет на результаты. Удалим его.

In [51]:
# Удалим значение
for i in data[data['gender'] == 'XNA'].index:
    data.drop(labels = [i],axis = 0, inplace = True)
# Обнулим индексы
data = data.reset_index(drop=True)

In [52]:
# Посмотрим на результат
column_analysis('gender')

Распределение значений в столбце gender:


F    14236
M     7288
Name: gender, dtype: int64

Строка удалена корректно. В датафрейме женщин в 2 раза больше, чем мужчин.

In [53]:
column_analysis('income_type')

Распределение значений в столбце income_type:


сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Данные корректны.

In [54]:
column_analysis('debt')

Распределение значений в столбце debt:


0    19783
1     1741
Name: debt, dtype: int64

Данные корректны. Более 91% клиентов не имели задолженностей по кредитам.

In [55]:
data[['total_income']].describe()

,total_income
count,2.152400e+04
mean,1.652235e+05
std,9.804559e+04
min,2.066726e+04
25%,1.077960e+05
50%,1.425944e+05
75%,1.955452e+05
max,2.265604e+06


По уровню доходов видно социальное неравенство. Большая часть клиентов получают меньше среднего (различие между средним и медианным значениями).

In [56]:
column_analysis('purpose')

Распределение значений в столбце purpose:


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Сразу видно большое количество похожих целей для получения кредита. Обработать эти дубликаты можно только с помощью лемматизации, что мы и сделаем позже.

Найдем и удалим все явные дубликаты.

In [57]:
# Попробуем найти явные дубликаты в таблице
print(f'Всего имеется {data.duplicated().sum()} дубликатов.')
# Выведем на экран 
data[data.duplicated()]

Всего имеется 71 дубликатов.


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1864.657692,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,покупка жилья для семьи
3290,0,13801.915694,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу
4182,1,1615.910188,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594.396847,свадьба
4851,0,14558.939054,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,свадьба
5557,0,13801.915694,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20701,0,14824.231775,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,дополнительное образование
21031,0,14558.939054,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,заняться образованием
21131,0,2203.078545,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,ремонт жилью
21280,1,1420.586863,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594.396847,покупка коммерческой недвижимости


In [58]:
print('Размер до удаления:', data.shape)
# Удалим дубликаты и обнулим индексы
data = data.drop_duplicates().reset_index(drop=True)
print('Размер после удаления:', data.shape)

Размер до удаления: (21524, 12)
Размер после удаления: (21453, 12)


**Вывод**

На данном этапе мы:
- произвели анализ столбцов;
- в столбце `children` заменили строки со значением -1 на 1, 20 на 2 ;
- определили, что в столбце `dob_year` есть 100 нулевых значений, которые никак не повлияют на ответы на вопросы;
- привели все значения столбцов `education` и `family_status` к нижнему регистру;
- проверили количественное соответствие в столбцах `education` и `family_status` столбцам `education_id` и `family_status_id`;
- удалили строку с единственным значением (XNA) столбца `gender`;
- нашли большое количесво похожих целей кредитов в столбце `purpose`, которые далее устраним с помощью лемматизации;
- удалили явные дубликаты.

### Замена типа данных

In [59]:
# Посмотрим на данные до замены типов
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Таблица data имеет:
- 2 столбца типа float64;
- 5 столбцов типа int64;
- 5 столбцов типа object.

Для упрощения работы с данными изменим тип данных в столбцах `days_employed` и `total_income`(условие задания).

In [60]:
# Изменим тип на int64
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [61]:
# Проверяем результат
data.info()
data.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21453 non-null  int64 
 1   days_employed     21453 non-null  int64 
 2   dob_years         21453 non-null  int64 
 3   education         21453 non-null  object
 4   education_id      21453 non-null  int64 
 5   family_status     21453 non-null  object
 6   family_status_id  21453 non-null  int64 
 7   gender            21453 non-null  object
 8   income_type       21453 non-null  object
 9   debt              21453 non-null  int64 
 10  total_income      21453 non-null  int64 
 11  purpose           21453 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6735,1,1593,41,среднее,1,женат / замужем,0,F,компаньон,1,115195,покупка жилья для сдачи
20744,0,4330,38,среднее,1,женат / замужем,0,F,компаньон,0,103146,строительство недвижимости
8306,0,958,24,среднее,1,женат / замужем,0,F,сотрудник,0,78408,свой автомобиль
12978,0,1232,43,среднее,1,гражданский брак,1,F,компаньон,0,217868,сыграть свадьбу
9419,0,1336,57,среднее,1,женат / замужем,0,F,компаньон,0,166681,строительство жилой недвижимости


**Вывод**

После изменения типов данных в  столбцах `days_employed` и `total_income` таблица data имеет только 2 типа данных - int64 и object.

### Лемматизация

С помощью лемматизации произведем оценку причин кредитования и их частоты. Создадим столбец `purpose_category`, в котором будем хранить категории кредитования.

In [62]:
# Импортируем библиотеку для лемматизации
from pymystem3 import Mystem
# Создадим список лемм для столбца purpose
lemmas_list = []
m = Mystem()
from collections import Counter
for purpose in data['purpose']:
# С помощью функций .join и .strip() склеим леммы и избавимся от пробелов и символов 
    lemmas = ''.join(m.lemmatize(purpose)).strip()
    lemmas_list.append(lemmas)
print(Counter(lemmas_list))

Counter({'автомобиль': 972, 'свадьба': 791, 'на проведение свадьба': 768, 'сыграть свадьба': 765, 'операция с недвижимость': 675, 'покупка коммерческий недвижимость': 661, 'операция с жилье': 652, 'покупка жилье для сдача': 651, 'операция с коммерческий недвижимость': 650, 'покупка жилье': 646, 'жилье': 646, 'покупка жилье для семья': 638, 'строительство собственный недвижимость': 635, 'недвижимость': 633, 'операция со свой недвижимость': 627, 'строительство жилой недвижимость': 624, 'покупка недвижимость': 620, 'покупка свой жилье': 620, 'строительство недвижимость': 619, 'ремонт жилье': 607, 'покупка жилой недвижимость': 606, 'на покупка свой автомобиль': 505, 'заниматься высокий образование': 496, 'сделка с подержанный автомобиль': 486, 'на покупка подержать автомобиль': 478, 'свой автомобиль': 478, 'на покупка автомобиль': 471, 'приобретение автомобиль': 461, 'дополнительный образование': 460, 'сделка с автомобиль': 455, 'высокий образование': 452, 'образование': 447, 'получение до

Самыми частовстречающимися леммами являются: свадьба, недвижимость, жильё, автомобиль, образование. Недвижимость и  жильё отнесем к одной категории недвижимость.

Создадим функцию для лемматизации. На вход подается строка, а на выход категория.

In [63]:
# Создадим функцию для лемматизации
def purpose_lemma(row):
    lem_purpose = m.lemmatize(row['purpose'])
    try:
        if 'автомобиль' in lem_purpose:
            return 'операции с автомобилем'
        elif 'свадьба' in lem_purpose:
            return 'проведение свадьбы'
        elif ('недвижимость' in lem_purpose) or ('жилье' in lem_purpose):
            return 'операции с недвижимостью'
        elif 'образование' in lem_purpose:
            return 'получение образования'   
    except:
        return 'unknown'
# Создадим новый столбец с категорией
data['purpose_category'] = data.apply(purpose_lemma, axis=1) 
# Проверим результат
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
914,0,16672,62,среднее,1,женат / замужем,0,F,пенсионер,0,115140,покупка своего жилья,операции с недвижимостью
12672,1,14632,57,высшее,0,в разводе,3,F,пенсионер,0,67346,на покупку подержанного автомобиля,операции с автомобилем
15711,0,2254,45,среднее,1,женат / замужем,0,F,компаньон,0,172357,заняться образованием,получение образования
21278,0,8255,52,среднее,1,вдовец / вдова,2,F,госслужащий,0,81624,покупка жилья,операции с недвижимостью
18841,1,2468,34,высшее,0,гражданский брак,1,F,компаньон,0,267887,операции с недвижимостью,операции с недвижимостью


С помощью лемматизации мы определили самые частые причины кредитования и создали столбец `purpose_category` с 4 категориями кредитования: свадьба, недвижимость, автомобиль, образование. В дальнейшем это упростит нам работу в ответе на вопрос про влияние цели кредита на его возврат.



### Удаление неинформативных столбцов

Удалим дублирующие столбцы `education_id`, `family_status_id`, `purpose`.

In [64]:
drop_columns = ['education_id', 'family_status_id', 'purpose']
for i in drop_columns:
    data = data.drop([i], axis=1)

In [65]:
# Посмотрим на результат
data.sample(5)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_category
18074,0,3566,33,высшее,не женат / не замужем,F,сотрудник,0,78796,операции с недвижимостью
17364,0,15377,65,среднее,гражданский брак,M,пенсионер,0,81710,операции с недвижимостью
5132,2,4488,35,высшее,женат / замужем,F,компаньон,0,259131,операции с недвижимостью
6429,0,14981,54,среднее,женат / замужем,F,пенсионер,0,97721,операции с недвижимостью
17361,0,1024,55,среднее,не женат / не замужем,M,сотрудник,0,151164,операции с недвижимостью


### Категоризация данных

Для ответа на вопросы нам нужны значения столбцов: `children`, `family_status`, `debt`, `total_income`, `purpose`.
Для упрощения анализа и ответа на вопросы произведем категоризацию значений столбцов: `children`, `total_income`. 

In [66]:
display(data['children'].value_counts())

0    14090
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Столбец `children` категоризируем по следующему принципу:

- 0 - детей нет;
- 1 и более - дети есть.

In [67]:
# Создадим функцию для категоризации детей
def children_categorization(row):
    count = row['children']
    try:
        if count == 0:
            return 'детей нет'
        elif 0 < count:
            return 'дети есть'
    except:
        return 'Такого не может быть. Проверь данные в столбце children.'
# Создадим новый столбец с категорией
data['children_category'] = data.apply(children_categorization, axis=1) 
# Проверим результат
display(data.sample(5))

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_category,children_category
14017,1,1827,46,среднее,женат / замужем,M,компаньон,0,130966,операции с автомобилем,дети есть
3712,0,6521,41,среднее,женат / замужем,F,сотрудник,0,101613,операции с автомобилем,детей нет
2998,0,1315,29,среднее,женат / замужем,F,сотрудник,0,142594,операции с автомобилем,детей нет
10610,0,14186,71,среднее,вдовец / вдова,F,пенсионер,0,43508,получение образования,детей нет
6800,0,8636,56,среднее,женат / замужем,F,компаньон,0,221809,операции с недвижимостью,детей нет


Столбец total_income категоризируем по следующему принципу:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [68]:
# Создадим функцию для категоризации дохода
def total_income_categorization(row):
    income = row['total_income']
    try:
        if income >= 1000001:
            return 'A'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 30001 <= income <= 50000:
            return 'D'
        elif income <= 30000:
            return 'E'
    except:
        return 'Такого не может быть. Проверь данные в столбце total_income.'
# Создадим новый столбец с категорией
data['total_income_category'] = data.apply(total_income_categorization, axis=1) 
display(data.sample(5))

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose_category,children_category,total_income_category
17904,0,289,22,высшее,гражданский брак,F,компаньон,0,277479,получение образования,детей нет,B
12915,1,435,22,среднее,женат / замужем,F,сотрудник,0,90522,операции с недвижимостью,дети есть,C
12718,1,4620,38,среднее,гражданский брак,F,сотрудник,0,146072,операции с недвижимостью,дети есть,C
18060,0,839,25,высшее,гражданский брак,F,компаньон,0,121571,проведение свадьбы,детей нет,C
4150,1,1315,29,высшее,женат / замужем,F,сотрудник,0,142594,операции с недвижимостью,дети есть,C


**Вывод**

Все необходимые столбцы категоризированы. Теперь можем перейти к ответу на вопросы.

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [69]:
# Сведем все полученные результат в одну таблицу
pivot_table_children = data.pivot_table(index='children_category', columns= 'debt', values='days_employed', aggfunc='count')
# Скорректируем отображаемое наименование столбцов
pivot_table_children.columns = ['Нет задолженности', 'Есть задолженность']
# Добавим новый столбец
pivot_table_children['Доля должников, в %'] = round(pivot_table_children['Есть задолженность'] / (pivot_table_children['Нет задолженности'] + pivot_table_children['Есть задолженность']) * 100, 2)
# Выведем итоговую таблицу
pivot_table_children

,Нет задолженности,Есть задолженность,"Доля должников, в %"
children_category,,,
детей нет,13027,1063,7.54
дети есть,6685,678,9.21


На основании таблицы можно сделать вывод, что клиенты без детей на 18% реже имеют задолженности по возврату кредита, чем клиенты с детьми.  

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [70]:
# Сведем все полученные результат в одну таблицу
pivot_table_family_status = data.pivot_table(index='family_status', columns= 'debt', values='days_employed', aggfunc='count')
# Скорректируем отображаемое наименование столбцов
pivot_table_family_status.columns = ['Нет задолженности', 'Есть задолженность']
# Добавим новый столбец
pivot_table_family_status['Доля должников, в %'] = round(pivot_table_family_status['Есть задолженность'] / (pivot_table_family_status['Есть задолженность'] + pivot_table_family_status['Нет задолженности']) * 100, 2)
# Выведем итоговую таблицу
pivot_table_family_status

,Нет задолженности,Есть задолженность,"Доля должников, в %"
family_status,,,
в разводе,1110,85,7.11
вдовец / вдова,896,63,6.57
гражданский брак,3762,388,9.35
женат / замужем,11408,931,7.55
не женат / не замужем,2536,274,9.75


**Вывод**

Семейное положение оказывает сильное влияние на наличие задолженности. Например, у вдовцов на 32 % меньше задолженностей по кредитах, чем у не женатых.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [71]:
# Сведем все полученные результат в одну таблицу
pivot_table_total_income_category = data.pivot_table(index='total_income_category', columns= 'debt', values='days_employed', aggfunc='count')
# Скорректируем отображаемое наименование столбцов
pivot_table_total_income_category.columns = ['Нет задолженности', 'Есть задолженность']
# Добавим новый столбец
pivot_table_total_income_category['Доля должников, в %'] = round(pivot_table_total_income_category['Есть задолженность'] / (pivot_table_total_income_category['Есть задолженность'] + pivot_table_total_income_category['Нет задолженности']) * 100, 2)
# Выведем итоговую таблицу
pivot_table_total_income_category

,Нет задолженности,Есть задолженность,"Доля должников, в %"
total_income_category,,,
A,23,2,8.00
B,4685,356,7.06
C,14655,1360,8.49
D,329,21,6.00
E,20,2,9.09


**Вывод**

Явной взаимосвязи между доходом и наличием задолженности нет.

- Как разные цели кредита влияют на его возврат в срок?

In [72]:
# Сведем все полученные результат в одну таблицу
pivot_table_purpose_category = data.pivot_table(index='purpose_category', columns= 'debt', values='days_employed', aggfunc='count')
# Скорректируем отображаемое наименование столбцов
pivot_table_purpose_category.columns = ['Нет задолженности', 'Есть задолженность']
# Добавим новый столбец
pivot_table_purpose_category['Доля должников, в %'] = round(pivot_table_purpose_category['Есть задолженность'] / (pivot_table_purpose_category['Есть задолженность'] + pivot_table_purpose_category['Нет задолженности']) * 100, 2)
# Выведем итоговую таблицу
pivot_table_purpose_category

,Нет задолженности,Есть задолженность,"Доля должников, в %"
purpose_category,,,
операции с автомобилем,3903,403,9.36
операции с недвижимостью,10028,782,7.23
получение образования,3643,370,9.22
проведение свадьбы,2138,186,8.00


**Вывод**

Цель кредитования оказывает влияние на наличие задолженности. Если цель кредита - операции с автомобилем, то вероятность задолженности самая высокая; если цель кредита - операции с недвижимостью, то вероятность задолженности самая низкая.

## Общий вывод

На основании произведенного анализа можно сделать вывод, что:
- клиенты без детей на 18% реже имеют задолженности по кредиту;
- семейное положение оказывает сильное влияние на наличие задолженности, например, у вдовцов на 32 % меньше задолженностей по кредитах, чем у не женатых;
- уровень доходов клиентов не влияет на возврат кредита в срок;
- цель кредитования оказывает влияние на наличие задолженности, например, самая высокая вероятность задолженности у кредита связанного с операциями с автомобилем, а самая низкая - у кредита связанного с операциями с недвижимостью.

Заказчику рекомендуется ввести фильтрацию вводимых значений для исключения ошибок в данных, а именно:
- `children` - не должно быть отрицательных значений, нужно дополнительно спрашивать, что значение введено верно, если оно привышает 5;
- `days_employed` - не должно быть отрицательных или нулевых значений; необходимо добавить проверку на реальность данных (days_employed <= dob_years * 365);
- `dob_years` - не должно быть нулевых значений и ниже определенного значения (например, 16 лет);
- `days_employed`, `total_income` необходимо внести контроль ввода данных;
- `education`, `family_status`, `gender`, `purpose` - можно ввести выбор из выпадающего списка, это позволит избежать разнообразия в написании.